In [37]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [38]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
from numpy import character
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."
    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

 # 首先输入玩家san值，以及投骰结果，返回掉了多少san值
def sanity_check_first(sanity):
    roll = d20.roll("1d100").total
    if roll < sanity:
        return 0
    return d20.roll("1d6").total

# 输入sanity的差值，以及玩家的智力，返回一个tag字符串（疯了），或者无返回（没疯）
def second(delta_sanity, intelligence):
    if delta_sanity >= 5 and d20.roll("1d100").total > intelligence:
        # go mad
        rand = random.randrange(0, 5)
        tags = [
        "fainting immediately",
        "incoherent speech",
        "uncontrollable twitching, trembling",
        "delusions of persecution",
        "strange appetites (dirt, clay, human flesh)",
        "scratches, punches, and bruises all over the body"
        ]
        return tags[rand]
    return None

# 输入sanity差值，以及tag，返回一段content描述
def third(delta_sanity, symptom):
    if delta_sanity == 0:
        return "I maintained my sanity."
    elif delta_sanity > 0 and tag is None:
        return "My sanity check failed, but I am not mad."
    elif delta_sanity > 0 and tag is not None:
         return f"My sanity check failed. I got a mad symptom: {symptom}"

    sanity == config.metadata ["stats"]["sanity"]
    print("sanity:", sanity)
    delta_sanity = first(sanity)  
    print("delta sanity:", delta_sanity)
    # character["sanity"] -= delta_sanity
    tag = second(delta_sanity, 45)
    print("tag:", tag)
    # character["tag"] += tag
    content = third(delta_sanity, tag)
    print("content:", content)
    new_sanity = sanity - delta_sanity
    print("new sanity:", new_sanity)

    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)

    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_cn_copy.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


你被一位富商雇佣，调查阿卡姆郊外的一座闹鬼的豪宅。
传说这座豪宅被诅咒，任何进入的人都不会再回来。
你已经得到了豪宅的地图和一些基本装备，以帮助你的调查。
你的目标是找出诅咒背后的真相，并结束它。

你站在闹鬼的豪宅前。
这座豪宅古老而破旧，墙上爬满了爬山虎，窗户被钉上了木板。
前门微微敞开，邀请你进入。

你要做什么？

possible actions:
1. 进入豪宅。(需要理智检定）
2. 拍摄豪宅外部的照片。
3. 与管家交谈。(可能需要一些说服力，以便让你获取更多信息)


I select option 1. TURN: 1 


在你决定进入这座阴森的豪宅之前，你感到一阵不祥的预感，让你的心跳加速。你知道，这将是一次对你勇气和理智的考验。

{由于决定进入豪宅，你需要进行一次理智检定，以确保你的角色能够保持冷静，面对即将到来的未知恐惧。}
skill: sanity
difficulty: medium